<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Figure 2: Comparing impact-specific measures of performance

_Authors: Oriana Chegwidden and Jeremy Freeman_

The methods below conduct the analyses to recreate Figure 2 included the
manuscript <<< insert doi >>>.


In [24]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy as sp
from carbonplan_forest_risks import load, setup, plot, fit, utils, prepare, collect
import xarray as xr
from carbonplan_forest_risks.utils import get_store
import altair as alt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import warnings

warnings.filterwarnings("ignore")

In [26]:
coarsen = 4

store = "az"
tlim = ("1984", "2018")

## Load in the fire timeseries data

#### Observations


In [27]:
mask = mask = (
    load.nlcd(store=store, year=2001).sel(band=[41, 42, 43, 90]).sum("band")
    > 0.25
).astype("float")

In [28]:
nlcd = load.nlcd(store=store, year=2001, coarsen=coarsen, mask=mask)
nftd = load.nftd(store=store, area_threshold=1500, coarsen=coarsen, mask=mask)

In [29]:
mtbs = load.mtbs(store=store, coarsen=coarsen, tlim=tlim, mask=mask)
# give it the same x/y coords as nftd (and nlcd and climate)
# this re-assignment will help us with some masking lower down
mtbs = mtbs.assign_coords({"x": nftd.x, "y": nftd.y})

In [30]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

#### Historical


In [31]:
historical_ds = xr.open_zarr(
    get_store("carbonplan-scratch", "data/fire_historical_v3.zarr")
).assign_coords({"x": nftd.x, "y": nftd.y})

In [32]:
model_active = ~np.isnan(
    historical_ds.isel(time=0).drop(["time"])["historical"]
)

## Load in the drought and biotic agent forest type analyses


#### Observations


#### Historical simulation


## Plot the joint figure


In [33]:
figure = alt.hconcat()
fire_ts = plot.fire.evaluation(
    mtbs["monthly"],
    historical_ds["historical"],
    model_active,
    clim=(0.0001, 0.005),
    add_map=False,
)
figure |= fire_ts

In [34]:
figure.configure_axis(grid=False)

alt.HConcatChart(...)